In [25]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.utils import resample
from bayes_opt import BayesianOptimization
import time


In [102]:
X_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/X_train_TS25_RS3.csv')
X_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/X_test_TS25_RS3.csv')
y_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/y_train_TS25_RS3.csv')
y_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/y_test_TS25_RS3.csv')

In [103]:
Narrow_X_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_X_train_TS25_RS3.csv')
Narrow_X_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_X_test_TS25_RS3.csv')
Narrow_y_train = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_y_train_TS25_RS3.csv')
Narrow_y_test = pd.read_csv('C:/Users/Prathmun/Documents/Springboard Jupyter/Capstone 2/DataDataCapstone/derived_data/Narrow_y_test_TS25_RS3.csv')

In [28]:
print('problemtype: medical : Catch all isntances of event, false positives are less of a problem. Maximize: Sensitivity, and Precision')
print('problemtype: spam: Catch many instances of event, but minimize FP')
print('problem type: Artisinal, run time less important than accuracy')
print('problem type: Enterprise, run time is a priority')

problemtype: medical : Catch all isntances of event, false positives are less of a problem. Maximize: Sensitivity, and Precision
problemtype: spam: Catch many instances of event, but minimize FP
problem type: Artisinal, run time less important than accuracy
problem type: Enterprise, run time is a priority


In [104]:
def timing_buddy(model_to_fit, dictionary_string, dicto):
    funky_time_start = time.time()
    model_to_fit.fit(X_train, np.ravel(y_train))
    funky_time_stop = time.time()
    funky_train_time = funky_time_stop - funky_time_start
    dicto[dictionary_string]['training_time'] = funky_train_time
    

In [105]:
def confused_buddy(model_to_confuse, dictionary_string,dicto):
    confuse = confusion_matrix(y_test, model_to_confuse.predict(X_test))
    dicto[dictionary_string]['confuse'] = confuse

In [ ]:
def recall_calculator(confuse)
    recall = 0
    tp = confuse[1][1]
    fn = confuse[1][0]
        if tp > 0 or tp ==1:
            recall = tp / (tp+fn)
    return recall

## Dummy

In [106]:
def classy_dummy(dicto):
    dummy_clf = DummyClassifier(strategy="most_frequent")
    timing_buddy(dummy_clf, 'dummy', dicto)
    confused_buddy(dummy_clf, 'dummy', dicto)
    confused_buddy(dummy_clf, 'dummy', dicto)
    dicto['dummy']['ROC_AUC_Score'] = roc_auc_score(y_test, dummy_clf.predict_proba(X_test)[:, 1])
    

## SVC

In [107]:
def SVCmodeler(dicto):
    SupportVectorClassifier = SVC()
    timing_buddy(SupportVectorClassifier, 'SVC', dicto)
    confused_buddy(SupportVectorClassifier, 'SVC', dicto)
    dicto['SVC']['ROC_AUC_Score'] = roc_auc_score(y_test, SupportVectorClassifier.decision_function(X_test))


## SGD Hyper Parameter optimization

In [ ]:
def SGDHyperParameterer(loss_penalty_list):
    SGD_clf = SGDClassifier(loss = loss_penalty_list[0], penalty=loss_penalty_list[1])
    SGD_clf.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, SGD_clf.predict(X_test))
    recall = recall_calculatorconfuse(confuse)
    return recall
    
    

In [ ]:
losses= ['modified_huber', 'log']
penalties = ['l2', 'l1', 'elasticnet']
param_grid = {}
count = 0
for each_loss in losses:
    for each_penalty in penalties:
        param_grid[str(each_loss) + '-' + str(each_penalty)] = [each_loss, each_penalty]
        count+=1

In [ ]:
SGD_Hyper_param_recall_scores = {}
optimization_start = time.time()
for key, value in param_grid.items():
    SGD_Hyper_param_recall_scores[key] = SGDHyperParameterer(value)
optimization_end = time.time()
optimization_time = (optimization_end - optimization_start)/60


    
    

In [ ]:
print('total optimization time: ' +str(optimization_time))
for key, value in SGD_Hyper_param_recall_scores.items():
    print(key + ' : ' +str(value))

## Stochastic Gradiant Descent 

In [38]:
def SGDModeler(dicto):
    SGD_clf = SGDClassifier(loss = 'modified_huber', penalty= 'l1"')
    timing_buddy(SGD_clf, 'SGDClassfier', dicto)
    confused_buddy(SGD_clf, 'SGDClassfier', dicto)
    dicto['SGDClassfier']['ROC_AUC_Score'] = roc_auc_score(y_test, SGD_clf.decision_function(X_test))

In [125]:
def grid_sgd_model(dicto):
    
    def SGDHyperParameterer(loss_penalty_list):
        SGD_clf = SGDClassifier(loss = loss_penalty_list[0], penalty=loss_penalty_list[1])
        SGD_clf.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, SGD_clf.predict(X_test))
        tp = confuse[1][1]
        fn = confuse[0][0]
        recall = tp / (tp+fn)
        return recall
    
    
    losses= ['modified_huber', 'log']
    penalties = ['l2', 'l1', 'elasticnet']
    param_grid = {}
    count = 0
    for each_loss in losses:
        for each_penalty in penalties:
            param_grid[str(each_loss) + '-' + str(each_penalty)] = [each_loss, each_penalty]
            count+=1

    
    ###
    SGD_Hyper_param_recall_scores = {}
    for key, value in param_grid.items():
        SGD_Hyper_param_recall_scores[key] = SGDHyperParameterer(value)

    ###
    best_recall = 0
    penalty = ''
    loss = ''
    for key, value in SGD_Hyper_param_recall_scores.items():
        if value > best_recall:
            best_recall = value
            loss, penalty = key.split('-')
            
        
                                                                 
    
    ###
    SGD_clf = SGDClassifier(loss = loss, penalty= penalty)
    timing_buddy(SGD_clf, 'SGDClassfier', dicto)
    confused_buddy(SGD_clf, 'SGDClassfier', dicto)
    dicto['SGDClassfier']['ROC_AUC_Score'] = roc_auc_score(y_test, SGD_clf.decision_function(X_test))

## bagging hyper parameter exploration

In [39]:
def bagginses_hyper(max_samples,max_features,n_estimators):
    Classy_Baggins_DT = BaggingClassifier(max_samples= max_samples,max_features = max_features, n_estimators=int(n_estimators))
    Classy_Baggins_DT.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, Classy_Baggins_DT.predict(X_test))
    tp = confuse[1][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    return recall
    

In [40]:
pparams ={ 'max_samples': (.1,.9),
       'max_features' : (.1,.9),
       'n_estimators': (10,500)}



In [41]:
optimizer = BayesianOptimization(
    bagginses_hyper,
    pbounds=pparams
)

In [42]:
bayes_sampling_time_start= time.time()
optimizer.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.000274 |  0.1161   |  0.4642   |  17.76    |
|  2        |  0.005107 |  0.2683   |  0.4594   |  53.77    |
|  3        |  0.05671  |  0.837    |  0.7867   |  117.4    |
|  4        |  0.0383   |  0.5987   |  0.3097   |  368.6    |
|  5        |  0.04584  |  0.6814   |  0.5042   |  427.2    |
|  6        |  0.02734  |  0.4747   |  0.854    |  261.7    |
|  7        |  0.04349  |  0.6602   |  0.4341   |  216.7    |
|  8        |  0.0      |  0.1332   |  0.1136   |  305.4    |
|  9        |  0.000182 |  0.1936   |  0.8871   |  265.5    |
|  10       |  0.05073  |  0.7833   |  0.1699   |  180.9    |
|  11       |  0.0      |  0.1082   |  0.3144   |  202.1    |
|  12       |  0.0      |  0.107    |  0.7576   |  428.5    |
|  13       |  0.04313  |  0.6184   |  0.8662   |  326.0    |
|  14       |  0.001826 |  0.2528   |  0.6588   |  178.9    |
|  15   

In [43]:
print(bayes_sampling_time_stop- bayes_sampling_time_start)

3100.323912858963


## BaggingClassifier

In [44]:
def BagginsesModeler(dicto):
    Classy_Baggins_DT = BaggingClassifier(n_estimators=12, max_features=0.7115, max_samples =0.689 )
    timing_buddy(Classy_Baggins_DT, 'BaggingClassifier', dicto)
    confused_buddy(Classy_Baggins_DT, 'BaggingClassifier', dicto)
    dicto['BaggingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, Classy_Baggins_DT.predict_proba(X_test)[:, 1])

In [130]:
def bayes_bagging_model(dicto):
    
    def bagginses_hyper(max_samples,max_features,n_estimators):
        Classy_Baggins_DT = BaggingClassifier(max_samples= max_samples,max_features = max_features, n_estimators=int(n_estimators))
        Classy_Baggins_DT.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, Classy_Baggins_DT.predict(X_test))
        tp = confuse[1][1]
        fn = confuse[0][0]
        recall = tp / (tp+fn)
        return recall


    
    ###
    pparams ={ 'max_samples': (.1,.9),
       'max_features' : (.1,.9),
       'n_estimators': (10,500)}

    ###
    
    optimizer = BayesianOptimization(
    bagginses_hyper,
    pbounds=pparams)
    
    ######
    optimizer.maximize(init_points=15, n_iter=10)
    bagginses_params= optimizer.max
    bagginses_params = bagginses_params['params']
    
    ###
    Classy_Baggins_DT = BaggingClassifier(n_estimators=int(bagginses_params['n_estimators']), 
                                          max_features=bagginses_params['max_features'],
                                          max_samples =bagginses_params['max_samples'] )
    timing_buddy(Classy_Baggins_DT, 'BaggingClassifier', dicto)
    confused_buddy(Classy_Baggins_DT, 'BaggingClassifier', dicto)
    dicto['BaggingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, Classy_Baggins_DT.predict_proba(X_test)[:, 1])

## random forest bayes hyper parameter tuning

In [45]:
model_set = ['dummy','SVC', 'SGDClassfier', 'BaggingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'HistGradientBoostingClassifier']
dicto = {}
for each in model_set:
    dicto[each] =a{'confuse' : [], 'training_time' : 0, 'ROC_AUC_Score': 0}

In [46]:
def bayes_RFModeler_auto(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    RandoForest = RandomForestClassifier(n_estimators=int(n_estimators), max_features='auto', max_depth=int(max_depth),
                                         min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf))
    RandoForest.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, RandoForest.predict(X_test))
    
    tp = confuse[1][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    
    return recall

In [47]:
def bayes_RFModeler_sqrt(n_estimators, max_depth, min_samples_split, min_samples_leaf):
    RandoForest = RandomForestClassifier(n_estimators=int(n_estimators), max_features='sqrt', max_depth=int(max_depth),
                                         min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf))
    RandoForest.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, RandoForest.predict(X_test))
    tp = confuse[1][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)

    return recall

In [48]:
# Number of trees in random forest
param_dicts = { 'n_estimators' : (100, 2000),
              'max_depth' : (10,110),
              'min_samples_split': (2,10),
              'min_samples_leaf' : (1,5),
              }
#'max_features' : ('auto', 'sqrt'),


print(param_dicts)

{'n_estimators': (100, 2000), 'max_depth': (10, 110), 'min_samples_split': (2, 10), 'min_samples_leaf': (1, 5)}


In [49]:
optimizer_auto = BayesianOptimization(
    bayes_RFModeler_auto,
    pbounds=param_dicts
)

In [50]:
bayes_sampling_time_start= time.time()
optimizer_auto.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.0453   |  92.06    |  2.685    |  9.067    |  1.044e+0 |
|  2        |  0.04135  |  39.33    |  4.551    |  9.342    |  1.902e+0 |
|  3        |  0.02957  |  15.4     |  2.447    |  6.976    |  776.5    |
|  4        |  0.04479  |  86.65    |  2.896    |  5.125    |  559.6    |
|  5        |  0.04125  |  67.27    |  4.932    |  6.715    |  1.824e+0 |
|  6        |  0.04486  |  88.27    |  2.49     |  6.158    |  865.9    |
|  7        |  0.04245  |  25.01    |  1.685    |  6.961    |  351.1    |
|  8        |  0.04417  |  71.17    |  2.401    |  9.247    |  135.4    |
|  9        |  0.01664  |  12.41    |  3.334    |  4.688    |  1.906e+0 |
|  10       |  0.04708  |  81.93    |  1.254    |  4.309    |  1.824e+0 |
|  11       |  0.04739  |  104.3    |  1.221    |  7.019    |  1.102e+0 |
|  12       |  0.04091  |  31.47    | 

In [51]:
print(bayes_sampling_time_stop - bayes_sampling_time_start)

4045.3972730636597


In [52]:
optimizer_sqrt = BayesianOptimization(
    bayes_RFModeler_sqrt,
    pbounds=param_dicts
)

In [53]:
bayes_sampling_time_start= time.time()
optimizer_sqrt.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.04111  |  109.1    |  4.063    |  2.341    |  1.327e+0 |
|  2        |  0.04291  |  102.1    |  3.482    |  5.554    |  1.091e+0 |
|  3        |  0.04316  |  54.09    |  3.781    |  2.746    |  737.0    |
|  4        |  0.04416  |  74.41    |  2.658    |  5.963    |  123.0    |
|  5        |  0.0417   |  81.43    |  4.987    |  5.447    |  225.1    |
|  6        |  0.0428   |  75.48    |  3.223    |  4.374    |  850.1    |
|  7        |  0.04082  |  93.69    |  4.212    |  4.672    |  1.068e+0 |
|  8        |  0.04175  |  95.66    |  4.587    |  5.097    |  185.9    |
|  9        |  0.04362  |  26.51    |  1.449    |  3.275    |  1.229e+0 |
|  10       |  0.04706  |  75.83    |  1.808    |  4.042    |  1.319e+0 |
|  11       |  0.04092  |  67.94    |  4.068    |  4.104    |  1.822e+0 |
|  12       |  0.02011  |  13.79    | 

In [54]:
print((bayes_sampling_time_stop - bayes_sampling_time_start)/60)

44.400212434927624


In [ ]:
## RandomForestClassifier

In [ ]:
def RFModeler(dicto):
    RandoForest = RandomForestClassifier(max_depth=82, min_samples_split = 2, min_samples_leaf=3, max_features='auto', n_estimators=228)
    
    timing_buddy(RandoForest, 'RandomForestClassifier', dicto)
    confused_buddy(RandoForest, 'RandomForestClassifier', dicto)
    dicto['RandomForestClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, RandoForest.predict_proba(X_test)[:, 1])

In [143]:
def bayes_rf_model(dicto):
    
    def bayes_RFModeler_sqrt(n_estimators, 
                             max_depth,
                             min_samples_split,
                             min_samples_leaf):
        RandoForest = RandomForestClassifier(n_estimators=int(n_estimators), max_features='sqrt', max_depth=int(max_depth),
                                             min_samples_split=int(min_samples_split), min_samples_leaf=int(min_samples_leaf))
        RandoForest.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, RandoForest.predict(X_test))
        tp = confuse[1][1]
        fn = confuse[0][0]
        recall = tp / (tp+fn)

        return recall

    
    ###
    param_dicts = { 'n_estimators' : (100, 2000),
              'max_depth' : (10,60),
              'min_samples_split': (2,10),
              'min_samples_leaf' : (1,5),
              }
    ###
    
    optimizer = BayesianOptimization(
    bayes_RFModeler_sqrt,
    pbounds=param_dicts)
    
    ######
    optimizer.maximize(init_points=15, n_iter=10)
    rf_params= optimizer.max
    rf_params = rf_params['params']
    
    ###
    RandoForest = RandomForestClassifier(n_estimators=int(rf_params['n_estimators']), 
                                         max_features='sqrt',
                                         max_depth=int(rf_params['max_depth']),
                                         min_samples_split=int(rf_params['min_samples_split']),
                                         min_samples_leaf=int(rf_params['min_samples_leaf']))
    
    timing_buddy(RandoForest, 'RandomForestClassifier', dicto)
    confused_buddy(RandoForest, 'RandomForestClassifier', dicto)
    dicto['RandomForestClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, RandoForest.predict_proba(X_test)[:, 1])

## AdaBoost Hyper Parameter Tuning

In [56]:
def AdaBoost_hyper(n_estimators, learning_rate):
    adaira_the_classifier = AdaBoostClassifier(learning_rate=learning_rate,  n_estimators=int(n_estimators))
    adaira_the_classifier.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, adaira_the_classifier.predict(X_test))
    tp = confuse[1][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    return recall
    

In [57]:
ada_params = {'learning_rate' : (.1,2),
              'n_estimators':(10,500),}

In [58]:
optimizer = BayesianOptimization(
    AdaBoost_hyper,
    pbounds=ada_params
)

In [59]:
bayes_sampling_time_start= time.time()
optimizer.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | learni... | n_esti... |
-------------------------------------------------
|  1        |  0.05732  |  1.877    |  134.1    |
|  2        |  0.05019  |  0.1287   |  306.0    |
|  3        |  0.006461 |  0.2626   |  15.28    |
|  4        |  0.05967  |  0.513    |  457.3    |
|  5        |  0.04827  |  1.947    |  109.4    |
|  6        |  0.0635   |  1.335    |  270.7    |
|  7        |  0.06133  |  1.018    |  194.7    |
|  8        |  0.05779  |  0.4215   |  357.8    |
|  9        |  0.06134  |  1.778    |  165.6    |
|  10       |  0.052    |  1.626    |  21.98    |
|  11       |  0.05301  |  0.2238   |  259.5    |
|  12       |  0.06245  |  1.61     |  291.1    |
|  13       |  0.06232  |  1.734    |  415.9    |
|  14       |  0.03671  |  0.141    |  98.62    |
|  15       |  0.06252  |  1.517    |  403.9    |
|  16       |  0.06241  |  1.211    |  269.2    |
|  17       |  0.05029  |  0.1      |  409.9    |
|  18       |  0.0629   |  1.502    |  398.2    |


In [60]:
print(bayes_sampling_time_stop- bayes_sampling_time_start)

1305.37842130661


## AdaBooastClassifier

In [61]:
def AdaBoostModeler(dicto):
    adaira_the_classifier = AdaBoostClassifier(n_estimators=48, learning_rate=1.278)
    timing_buddy(adaira_the_classifier, 'AdaBoostClassifier', dicto)
    confused_buddy(adaira_the_classifier, 'AdaBoostClassifier', dicto)
    dicto['AdaBoostClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, adaira_the_classifier.predict_proba(X_test)[:, 1])

In [95]:
def bayes_hyper_ada_boost_model(dicto):
    
    def AdaBoost_hyper(n_estimators, learning_rate):
        adaira_the_classifier = AdaBoostClassifier(learning_rate=learning_rate,  n_estimators=int(n_estimators))
        adaira_the_classifier.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, adaira_the_classifier.predict(X_test))
        tp = confuse[1][1]
        fn = confuse[0][0]
        recall = tp / (tp+fn)
        return recall

    
    ###
    ada_params = {'learning_rate' : (.1,2),
              'n_estimators':(10,500),}
    ###
    
    optimizer = BayesianOptimization(
    AdaBoost_hyper,
    pbounds=ada_params)
    
    ######
    optimizer.maximize(init_points=15, n_iter=10)
    ada_boost_params= optimizer.max
    ada_boost_params = ada_boost_params['params']
    
    ###
    adaira_the_classifier = AdaBoostClassifier(learning_rate=ada_boost_params['learning_rate'],  n_estimators=int(ada_boost_params['n_estimators']))
    timing_buddy(adaira_the_classifier, 'AdaBoostClassifier', dicto)
    confused_buddy(adaira_the_classifier, 'AdaBoostClassifier', dicto)
    dicto['AdaBoostClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, adaira_the_classifier.predict_proba(X_test)[:, 1])

## Gradiant Boosting Hyper Parameter Tuning

In [62]:
def GradBoost_hyper(n_estimators, learning_rate, max_leaf_nodes):
    grady_the_boosted = GradientBoostingClassifier(learning_rate=learning_rate,  n_estimators=int(n_estimators),
                                                  max_leaf_nodes=int(max_leaf_nodes))
    grady_the_boosted.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, grady_the_boosted.predict(X_test))
    tp = confuse[1][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    return recall
    

In [63]:
GradBoost_params = {'learning_rate' : (.1,2),
                    'n_estimators':(10,500),
                    'max_leaf_nodes':(3,50)}

#max features - strings, to be translated
#min_impurity - float


In [64]:
optimizer = BayesianOptimization(
    GradBoost_hyper,
    pbounds=GradBoost_params
)

In [65]:
bayes_sampling_time_start= time.time()
optimizer.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | learni... | max_le... | n_esti... |
-------------------------------------------------------------
|  1        |  0.06318  |  0.107    |  37.72    |  469.2    |
|  2        |  0.06662  |  1.06     |  40.7     |  71.06    |
|  3        |  0.0729   |  1.261    |  24.68    |  361.0    |
|  4        |  0.0719   |  0.5334   |  32.09    |  384.6    |
|  5        |  0.07009  |  1.472    |  45.35    |  373.5    |
|  6        |  0.06832  |  1.468    |  48.36    |  328.0    |
|  7        |  0.07203  |  1.02     |  18.99    |  230.8    |
|  8        |  0.07184  |  0.7948   |  42.6     |  238.1    |
|  9        |  0.06912  |  0.3871   |  29.58    |  356.8    |
|  10       |  0.06704  |  0.8176   |  23.21    |  46.39    |
|  11       |  0.06609  |  0.8764   |  4.852    |  178.6    |
|  12       |  0.06781  |  0.7167   |  8.559    |  86.48    |
|  13       |  0.06825  |  1.504    |  9.206    |  36.38    |
|  14       |  0.07048  |  0.6056   |  21.72    |  252.0    |
|  15   

In [66]:
print(bayes_sampling_time_stop- bayes_sampling_time_start)

1440.5923175811768


In [98]:
def bayes_hyper_grad_boosting_model(dicto):
    
    def GradBoost_hyper(n_estimators, learning_rate, max_leaf_nodes):
        grady_the_boosted = GradientBoostingClassifier(learning_rate=learning_rate,  n_estimators=int(n_estimators),
                                                      max_leaf_nodes=int(max_leaf_nodes))
        grady_the_boosted.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, grady_the_boosted.predict(X_test))
        tp = confuse[1][1]
        fn = confuse[0][0]
        recall = tp / (tp+fn)
        return recall
    
    ###
    GradBoost_params = {'learning_rate' : (.1,2),
                    'n_estimators':(10,500),
                    'max_leaf_nodes':(3,50)}
    ###
    
    optimizer = BayesianOptimization(
    GradBoost_hyper,
    pbounds=GradBoost_params)
    
    ######
    optimizer.maximize(init_points=15, n_iter=10)
    grad_boost_params= optimizer.max
    grad_boost_params = grad_boost_params['params']
    
    ###
    grady_the_boosted = GradientBoostingClassifier(learning_rate=grad_boost_params['learning_rate'],
                                                   n_estimators=int(grad_boost_params['n_estimators']),
                                                   max_leaf_nodes=int(grad_boost_params['max_leaf_nodes']))
    timing_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    confused_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    dicto['GradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, grady_the_boosted.predict_proba(X_test)[:, 1])
    ###

## GradientBoostingClassifier

def GradBoostModeler(dicto):
    grady_the_boosted = GradientBoostingClassifier(n_estimators=100)
    timing_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    confused_buddy(grady_the_boosted, 'GradientBoostingClassifier', dicto)
    dicto['GradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, grady_the_boosted.predict_proba(X_test)[:, 1])

## HistGradient Hyper Parameter Tuning

In [76]:
def histBoost_hyper(learning_rate, max_leaf_nodes):
    histamine_reaction = HistGradientBoostingClassifier(learning_rate=learning_rate,
                                                  max_leaf_nodes=int(max_leaf_nodes))
    histamine_reaction.fit(X_train, np.ravel(y_train))
    confuse = confusion_matrix(y_test, histamine_reaction.predict(X_test))
    tp = confuse[1][1]
    fn = confuse[0][0]
    recall = tp / (tp+fn)
    return recall
    

In [77]:
HistBoost_params = {'learning_rate' : (.1,2),
                    'max_leaf_nodes':(3,50)}

#max features - strings, to be translated
#min_impurity - float


In [78]:
optimizer = BayesianOptimization(
    histBoost_hyper,
    pbounds=HistBoost_params
)

In [79]:
bayes_sampling_time_start= time.time()
optimizer.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

|   iter    |  target   | learni... | max_le... |
-------------------------------------------------
|  1        |  0.06285  |  1.307    |  27.81    |
|  2        |  0.03947  |  1.516    |  16.55    |
|  3        |  0.07139  |  0.9505   |  44.23    |
|  4        |  0.06378  |  0.5472   |  15.57    |
|  5        |  0.06183  |  0.4077   |  46.66    |
|  6        |  0.0725   |  1.448    |  35.79    |
|  7        |  0.04512  |  0.109    |  3.687    |
|  8        |  0.06524  |  1.111    |  8.274    |
|  9        |  0.06625  |  1.507    |  10.75    |
|  10       |  0.07251  |  0.9713   |  47.93    |
|  11       |  0.06677  |  1.309    |  11.75    |
|  12       |  0.06047  |  0.5854   |  8.122    |
|  13       |  0.06293  |  1.421    |  7.635    |
|  14       |  0.06743  |  0.9802   |  16.32    |
|  15       |  0.06139  |  0.3      |  16.09    |
|  16       |  0.05874  |  1.48     |  27.9     |
|  17       |  0.07345  |  0.8875   |  47.7     |
|  18       |  0.06166  |  0.2993   |  41.15    |


In [80]:
print(bayes_sampling_time_stop- bayes_sampling_time_start)

48.84417653083801


In [82]:
hist_grad_params= optimizer.max

In [84]:
hist_grad_params = hist_grad_params['params']
hist_grad_params

{'learning_rate': 1.09469075035991, 'max_leaf_nodes': 37.16591160227114}

In [86]:
hist_grad_params['learning_rate']

37.16591160227114

In [153]:
def bayes_hyper_hist_grad_model(dicto):
    
    def histBoost_hyper(learning_rate, max_leaf_nodes):
        histamine_reaction = HistGradientBoostingClassifier(learning_rate=learning_rate,
                                                      max_leaf_nodes=int(max_leaf_nodes))
        histamine_reaction.fit(X_train, np.ravel(y_train))
        confuse = confusion_matrix(y_test, histamine_reaction.predict(X_test))
        tp = confuse[1][1]
        fn = confuse[0][0]
        recall = tp / (tp+fn)
        return recall
    ###
    HistBoost_params = {'learning_rate' : (.1,2),
                    'max_leaf_nodes':(3,50)}
    ###
    optimizer = BayesianOptimization(
    histBoost_hyper,
    pbounds=HistBoost_params)
    ######
    optimizer.maximize(init_points=15, n_iter=10)
    hist_grad_params= optimizer.max
    print(hist_grad_params)
    hist_grad_params = hist_grad_params['params']
    ###
    histamine_reaction = HistGradientBoostingClassifier(learning_rate =hist_grad_params['learning_rate'],
                                                        max_leaf_nodes= int(hist_grad_params['max_leaf_nodes']))
    hist_time_start = time.time()
    histamine_reaction.fit(X_train, np.ravel(y_train))
    hist_time_stop = time.time()
    hist_train_time = hist_time_stop - hist_time_start
    ###
    dicto['HistGradientBoostingClassifier']['training_time'] = hist_train_time
    confuse = confusion_matrix(y_test, histamine_reaction.predict(X_test))
    dicto['HistGradientBoostingClassifier']['confuse'] = confuse
    dicto['HistGradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, histamine_reaction.predict_proba(X_test)[:, 1])    

## HistGradientBoostingClassifier


In [154]:
#for some reason this model didn't like my functions. So long form it is!
def HistGrad_modeler(dicto):
    histamine_reaction = HistGradientBoostingClassifier()

    hist_time_start = time.time()
    histamine_reaction.fit(X_train, np.ravel(y_train))
    hist_time_stop = time.time()
    hist_train_time = hist_time_stop - hist_time_start
    dicto['HistGradientBoostingClassifier']['training_time'] = hist_train_time
    confuse = confusion_matrix(y_test, histamine_reaction.predict(X_test))
    dicto['HistGradientBoostingClassifier']['confuse'] = confuse
    dicto['HistGradientBoostingClassifier']['ROC_AUC_Score'] = roc_auc_score(y_test, histamine_reaction.predict_proba(X_test)[:, 1])    


## Model Evaluation

#precision = tp/(tp+fp)
maximize this for spam, don't wanna be hiding real emails

#recall= tp/(tp+fn) 
Maximize this for medical or security scenarios, don't wanna miss actual sicknesses

In [246]:
def medical_evaluator(dicto):
    bestmodel = ""
    training_time= 0
    recall = 0
    temp_recall = 0
    for key, value in dicto.items():
        tp = dicto[key]['confuse'][1][1]
        fn = dicto[key]['confuse'][1][0]
        if tp > 0 or tp ==1:
            temp_recall = tp / (tp+fn)

        if temp_recall > recall:

            recall = temp_recall
            bestmodel = key
            training_time = dicto[key]['training_time']
    return bestmodel, recall, training_time

In [242]:

print(medical_evaluator(wide_data_model_stats))

dummy
0
2671
SVC
471
2200
SGDClassfier
701
1970
BaggingClassifier
637
2034
RandomForestClassifier
527
2144
AdaBoostClassifier
706
1965
GradientBoostingClassifier
1859
812
HistGradientBoostingClassifier
1096
1575
('GradientBoostingClassifier', 0.6959940097341819, 59.56250500679016)


In [243]:
for key, value in wide_data_model_stats.items():
    print(key)
    print(" ")
    print(value['training_time'])
    print(value['confuse'])

dummy
 
0.07002544403076172
[[10938     0]
 [ 2671     0]]
SVC
 
397.05378580093384
[[10680   258]
 [ 2200   471]]
SGDClassfier
 
1.420055866241455
[[10489   449]
 [ 1970   701]]
BaggingClassifier
 
257.0642306804657
[[10434   504]
 [ 2034   637]]
RandomForestClassifier
 
133.97812008857727
[[10634   304]
 [ 2144   527]]
AdaBoostClassifier
 
21.616897344589233
[[10476   462]
 [ 1965   706]]
GradientBoostingClassifier
 
59.56250500679016
[[6569 4369]
 [ 812 1859]]
HistGradientBoostingClassifier
 
1.0342340469360352
[[9222 1716]
 [1575 1096]]


In [244]:
def spam_evaluator(dicto):
    bestmodel = ""
    training_time= 0
    precision = 0
    temp_precision = 0.00000001
    for key, value in dicto.items():

        tp = dicto[key]['confuse'][0][1]
        fp = dicto[key]['confuse'][0][1]
        if tp > 0 or tp == 1:
            temp_precision = tp/(tp + fp)

        if temp_precision > precision:

            precision = temp_precision
            bestmodel = key
            training_time = dicto[key]['training_time']
    return bestmodel, precision, training_time

In [190]:
def chonky_model_aggregator(X_train, y_train, X_test, y_test):
    
    #Models to explore
    model_set = ['dummy','SVC', 'SGDClassfier', 'BaggingClassifier', 'RandomForestClassifier', 'AdaBoostClassifier', 'GradientBoostingClassifier', 'HistGradientBoostingClassifier']
    model_stats = {}
    for each in model_set:
        model_stats[each] ={'confuse' : [], 'training_time' : 0, 'ROC_AUC_Score': 0}
    #model running
    classy_dummy(model_stats)
    SVCmodeler(model_stats)
    grid_sgd_model(model_stats)
    bayes_bagging_model(model_stats)
    bayes_rf_model(model_stats)
    bayes_hyper_ada_boost_model(model_stats)
    bayes_hyper_grad_boosting_model(model_stats)
    bayes_hyper_hist_grad_model(model_stats)


    #best_med_model, med_precision, med_time = medical_evaluator(model_stats)
    #best_spam_model, spam_recall, spam_time = spam_evaluator(model_stats)
    #print("The best medical model (Optimizing for precision) is " + best_med_model + " It's precision score was" + str(round(med_precision,3)) + " and it took " + str(round(med_time,2)) +" seconds to run.")
    #print("")
    #print("The best spam model (Optimizing for recall) is " + best_spam_model + " It's recall score was" +str(round(spam_recall, 3)) +" and it took " + str(round(spam_time, 2) +" seconds to run.")
    return model_stats



#Wide Data

In [191]:
wide_data_model_stats = chonky_model_aggregator(X_train, y_train, X_test, y_test)

|   iter    |  target   | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.05253  |  0.7571   |  0.8434   |  352.0    |
|  2        |  0.02358  |  0.4118   |  0.6222   |  431.0    |
|  3        |  0.05344  |  0.8147   |  0.3698   |  437.8    |
|  4        |  0.000365 |  0.2223   |  0.6694   |  405.1    |
|  5        |  0.03395  |  0.5086   |  0.694    |  128.7    |
|  6        |  0.05784  |  0.8993   |  0.8989   |  180.5    |
|  7        |  0.000274 |  0.2256   |  0.5942   |  363.2    |
|  8        |  0.05232  |  0.7543   |  0.8533   |  348.1    |
|  9        |  0.03178  |  0.5066   |  0.4812   |  395.9    |
|  10       |  0.04376  |  0.6564   |  0.535    |  349.6    |
|  11       |  0.03048  |  0.4717   |  0.7416   |  93.96    |
|  12       |  0.04016  |  0.6014   |  0.4327   |  324.4    |
|  13       |  0.03321  |  0.543    |  0.4172   |  450.3    |
|  14       |  0.03889  |  0.5515   |  0.2542   |  117.4    |
|  15   

|   iter    |  target   | learni... | max_le... | n_esti... |
-------------------------------------------------------------
|  1        |  0.06565  |  0.3196   |  12.35    |  254.5    |
|  2        |  0.06561  |  0.4406   |  30.11    |  150.6    |
|  3        |  0.08534  |  1.578    |  11.81    |  264.3    |
|  4        |  0.07635  |  0.9605   |  33.04    |  468.7    |
|  5        |  0.03974  |  1.882    |  28.31    |  220.8    |
|  6        |  0.0677   |  1.428    |  24.34    |  101.4    |
|  7        |  0.06188  |  0.4754   |  47.46    |  63.89    |
|  8        |  0.06277  |  0.123    |  25.17    |  424.1    |
|  9        |  0.06852  |  0.3763   |  48.99    |  346.1    |
|  10       |  0.06476  |  0.2305   |  27.49    |  389.3    |
|  11       |  0.06976  |  1.44     |  47.2     |  428.9    |
|  12       |  0.06388  |  1.348    |  27.93    |  266.5    |
|  13       |  0.07112  |  1.454    |  23.54    |  328.8    |
|  14       |  0.0631   |  0.1389   |  11.44    |  499.2    |
|  15   

In [248]:
print(medical_evaluator(wide_data_model_stats))

('GradientBoostingClassifier', 0.6959940097341819, 59.56250500679016)


In [ ]:
best_med_model, med_recall, med_time = medical_evaluator(wide_data_model_stats)
best_spam_model, spam_precision, spam_time = spam_evaluator(wide_data_model_stats)
print("The best medical model (Optimizing for recall) is " + best_med_model + " It's recall score was" + str(round(med_precision,3)) + " and it took " + str(round(med_time,2)) +" seconds to run.")
print("")
print("The best spam model (Optimizing for precision) is " + best_spam_model +
      " It's precision score was" +str(round(spam_precision, 3)) +" and it took " + str(round(spam_time, 2) ) +" seconds to run.")

In [200]:
for key, value in wide_data_model_stats.items():
    print(key)
    print(value['confuse'])

dummy
[[10938     0]
 [ 2671     0]]
SVC
[[10680   258]
 [ 2200   471]]
SGDClassfier
[[10489   449]
 [ 1970   701]]
BaggingClassifier
[[10434   504]
 [ 2034   637]]
RandomForestClassifier
[[10634   304]
 [ 2144   527]]
AdaBoostClassifier
[[10476   462]
 [ 1965   706]]
GradientBoostingClassifier
[[6569 4369]
 [ 812 1859]]
HistGradientBoostingClassifier
[[9222 1716]
 [1575 1096]]


#Narrow Data

In [ ]:
narrow_data_model_stats = chonky_model_aggregator(Narrow_X_train, Narrow_y_train, Narrow_X_test, Narrow_y_test)

|   iter    |  target   | max_fe... | max_sa... | n_esti... |
-------------------------------------------------------------
|  1        |  0.02732  |  0.4621   |  0.6021   |  178.9    |
|  2        |  0.008906 |  0.3411   |  0.4708   |  295.2    |
|  3        |  0.05588  |  0.884    |  0.5601   |  479.6    |
|  4        |  0.02948  |  0.4732   |  0.7141   |  170.7    |
|  5        |  0.01485  |  0.3601   |  0.1567   |  112.9    |
|  6        |  0.0329   |  0.5054   |  0.2649   |  127.1    |
|  7        |  0.04665  |  0.6872   |  0.6116   |  440.6    |
|  8        |  0.049    |  0.7011   |  0.8998   |  253.5    |
|  9        |  0.007547 |  0.3128   |  0.2174   |  294.3    |
|  10       |  0.03092  |  0.4635   |  0.6925   |  76.73    |
|  11       |  0.03131  |  0.5173   |  0.2153   |  36.98    |
|  12       |  0.05339  |  0.8115   |  0.4941   |  414.8    |
|  13       |  0.01107  |  0.3462   |  0.5768   |  384.0    |
|  14       |  0.0      |  0.1605   |  0.3561   |  327.0    |
|  15   

In [ ]:
print(medical_evaluator(narrow_data_model_stats))

In [ ]:
best_med_model, med_recall, med_time = medical_evaluator(narrow_data_model_stats)
best_spam_model, spam_precision, spam_time = spam_evaluator(narrow_data_model_stats)
print("The best medical model (Optimizing for recall) is " + best_med_model + " It's recall score was" + str(round(med_precision,3)) + " and it took " + str(round(med_time,2)) +" seconds to run.")
print("")
print("The best spam model (Optimizing for precision) is " + best_spam_model +
      " It's precision score was" +str(round(spam_precision, 3)) +" and it took " + str(round(spam_time, 2) ) +" seconds to run.")

In [ ]:
for key, value in narrow_data_model_stats.items():
    print(key)
    print(value['confuse'])

In [ ]:
#That didn't go well, so we're going to play with oversampling our smokers in both sets

In [ ]:
Xy_to_oversample= X_train.join(y_train)

In [ ]:
Xy_mask = Xy_to_oversample.smokes == 'yes'
Xy_oversampled = resample(Xy_to_oversample[Xy_mask], n_samples=20000, replace=True)

In [ ]:
oversampled_X_train = Xy_oversampled.drop('smokes', axis=1)    
oversampled_y_train = Xy_oversampled.smokes

# resampled Wide Data

In [ ]:
wide_data_model_stats = chonky_model_aggregator(oversampled_X_train, oversampled_y_train, X_test, y_test)

In [ ]:
for key, value in wide_data_model_stats.items():
    print(key)a
    print(value['confuse'])

# Resampled Narrow Data

In [ ]:
Narrow_Xy_to_oversample= Narrow_X_train.join(Narrow_y_train)

In [ ]:
Narrow_Xy_mask = Narrow_Xy_to_oversample.smokes == 'yes'
Narrow_Xy_oversampled = resample(Narrow_Xy_to_oversample[Narrow_Xy_mask], n_samples=20000, replace=True)

In [ ]:
Narrow_oversampled_X_train = Narrow_Xy_oversampled.drop('smokes', axis=1)    
Narrow_oversampled_y_train = Narrow_Xy_oversampled.smokes

In [ ]:
narrow_resampled_data_model_stats = chonky_model_aggregator(Narrow_oversampled_X_train, Narrow_oversampled_y_train, Narrow_X_test, Narrow_y_test)

In [ ]:
for key, value in narrow_resampled_data_model_stats.items():
    print(key)
    print(value['confuse'])

# Bayes Resampling

In [ ]:
pbounds = {'samples': (500,50000)}

In [ ]:
def precise_sampling_bayes_box(samples):
    
    Xy_to_oversample= X_train.join(y_train)
    Xy_mask = Xy_to_oversample.smokes == 'yes'
    Xy_oversampled = resample(Xy_to_oversample[Xy_mask], n_samples=int(samples), replace=True)
    oversampled_X_train = Xy_oversampled.drop('smokes', axis=1)    
    oversampled_y_train = Xy_oversampled.smokes
    
    
    
    
    #testing the data
    model_stats = chonky_model_aggregator(oversampled_X_train, oversampled_y_train, X_test, y_test)
    best_spam_model, spam_precision, spam_time = spam_evaluator(model_stats)
    return med_precision
    

In [ ]:
def recall_sampling_bayes_box(samples):
    
    Xy_to_oversample= X_train.join(y_train)
    Xy_mask = Xy_to_oversample.smokes == 'yes'
    Xy_oversampled = resample(Xy_to_oversample[Xy_mask], n_samples=int(samples), replace=True)
    oversampled_X_train = Xy_oversampled.drop('smokes', axis=1)    
    oversampled_y_train = Xy_oversampled.smokes
    
    
    
    
    #testing the data
    model_stats = chonky_model_aggregator(oversampled_X_train, oversampled_y_train, X_test, y_test)
    best_med_model, recall, med_time = medical_evaluator(model_stats)
    return med_precision
    

In [ ]:
optimizer = BayesianOptimization(
    recall_sampling_bayes_box,
    pbounds=pbounds
)

In [ ]:
bayes_sampling_time_start= time.time()
optimizer.maximize(init_points=15, n_iter=10)
bayes_sampling_time_stop= time.time()

In [ ]:
print(((bayes_sampling_time_stop- bayes_sampling_time_start)/60)/60)